In [1]:
# Due to merge conflict this is now the notebook for the 3d case

In [2]:
# Pick convention: 
# We work on a rectangular cell of side lengths a, b, c
# This cuboid is in the positive coordinates and centered at 0
# I.e. spanned by (a,0,0), (0,b,0) and (0,0,c)

In [4]:
import numpy as np
import random as rd

## Generating random points (optional)

In [5]:
def gen_points(N=5,a=1,b=1,c=1):
    """
    Generates N random points in [0,a]x[0,b]x[0,c] cuboid,
    outputts Nx3 numpy array, each row containing x, y,z coordinates of a point.
    """
    return np.array([[rd.random(),rd.random(),rd.random()] for i in range(N)])


In [6]:
gen_points().shape

(5, 3)

## Duplicating points to neighbouring cells

In [ ]:
def torus_copy(points, a=1,b=1,c=1):
    """
    Takes numpy array points of N points on axbxc cube
    and creates 8 copies surrounding it.
    
    Returns new_points, containing original points and 9+8+9=26 offset copies,
    so Nx27 points in total.
    """
    
    N = np.shape(points)[0]
    # first index  = which axbxc cuboid we are working on
    # second index = index of corresponding original point
    # third index  = x, y, z coordinates of point
    new_points = np.zeros((27,N,3)) 

    i = 0 # to index the cuboid
    for x in [-a,0,a]:
        for y in [-b,0,b]:
            for z in [-c,0,c]:
            transl = np.array([[x,y,z] for dummy in range(N)])
            new_points[i,:,:] = points + transl
            i += 1
            
    return new_points.reshape(-1,3)

## Generating $\alpha$-filtration

In [ ]:

def dim_split(filt):
    """
    Takes list of simplices 'filt' with filtration values, 
    i.e. with elements ([3,2], 0.123)
    Splits filt by dimension of elements.
    Outputs the 4 seperate lists, one for each dimension 0 to 3.
    """
    simp_0 = []
    simp_1 = []
    simp_2 = []
    simp_3 = []
    
    for (simp, val) in filt:
        p = len(simp)-1
        
        if p == 0:
            simp_0.append((simp,val))
        if p == 1:
            simp_1.append((simp,val))
        if p == 2:
            simp_2.append((simp,val))
        if p == 3:
            simp_3.append((simp,val))
    return simp_0, simp_1, simp_2, simp_3

In [ ]:
def check_bound(point, a=1, b=1, c=1):
    """
    Takes point with x,y,z coordinates.
    Checks if point lies in cuboid [0,a) x [0,b) x [0,c).
    If yes, outputs True, else False.
    """
    x = point[0]
    y = point[1]
    z = point[2]
    
    return (    x >= 0 and x < a 
            and y >= 0 and y < b 
            and z >= 0 and z < 1)

In [ ]:
def equiv_num(x1,x2,m=1):
    """
    Calculates 'x1-x2 mod m', where we always want to take 
    the representative with the smallest absolute value. 
    """
    z1 = abs((x1 - x2)%a)
    z2 = abs(z1 - a)
    return min(z1,z2)

In [ ]:
def identification_list(S0_list, S0, eps = 1e-5):
    """
    Input:
        S0_list ... naive filtration list  of 0-simplices ([3],3) as given by gudhi
        S0      ... list of Simplex0D objects generated from S0

    Output:
        identify_list ... list of lists with entries [i,j], 
                           meaning that the ith vertex has to be matched to the jth Simplex0D object
    """
    # build identification list for later reference when building higher simplices
    identify_list = []

    for i in range(len(S0_list)):
        simp, filt_value = S0_list[i]
        coord = np.array(alpha_complex.get_point(i))

        for Simplex in S0: # looking for the point in the unit cell this corresponds to              
            other_coord = Simplex.coords

            if ((equiv_num(coord[0],other_coord[0],m=a)<eps) and 
                (equiv_num(coord[1],other_coord[1],m=b)<eps) and
                (equiv_num(coord[2],other_coord[2],m=c)<eps)):

                identify_index = Simplex.int_filt

        identify_list.append([i, identify_index])
    return identify_list


In [ ]:
def create_S0(S0_list):
    """
    Input:
        S0_list ... List of 0-simplices as generated by gudhi, 
        after having split them by dimension
    Output: 
        S0 ... List of Simplex0D objects containing the combinatorical representation,
        the integer filtration value, the continuous filtration value 
        and the geometric coordinate of the point
    """

    int_filt_value = 0
    S0 = []

    for i in range(len(simp0)):
        simp, filt_value = simp0[i]

        # check if 0-simplex lies in main cell
        coord = alpha_complex.get_point(i)
        if check_bound(coord,a=a,b=b,c=c): # point lies inside main cell
            S0.append(sc.Simplex0D([int_filt_value], 
                                            int_filt_value, 
                                            filt_value, 
                                            coord))
            int_filt_value +=1
    return S0

In [ ]:
def create_S1(S1_list,S0, alpha_complex):
    """
    Input: 
        Input:
        S1_list ... List of 1-simplices as generated by gudhi, 
        after having split them by dimension
    Output: 
        S1 ... List of Simplex1D objects containing the combinatorical representation,
        the integer filtration value, the continuous filtration value,
        the crossing vector as numpy array and the vertices constituting the boundary
        in lexicographical order
    """

    # note that this is not the actual integer filtration value 
    # and this will get corrected later on
    int_filt_value = len(S0) 
    S1 = []


    for i in range(len(simp1)):
        """
        This does not work in 3d, because for a total order we need more than left-right
        !!!!!!!
        """
        simp, filt_value = simp1[i]

        # check what the leftmost vertex is
        n0 = simp[0]
        n1 = simp[1]
        coord0 = alpha_complex.get_point(n0)
        coord1 = alpha_complex.get_point(n1)

        nl, coordl, nr, coordr = order_vertex(n0, coord0, n1, coord1)

        # Check if leftmost vertex is in unit cell
        """
        function "check_bound" has been modified
        this STILL needs to be adjusted here
        also modify for 3d-version
        """
        if check_bound(coordl, xmax=a, ymax=b):
            # Calculate crossing vector
            cross_vec = crossing_vector(coordr,xmax=a,ymax=b)

            # identify the old vertices with the newly numerated ones!!!!! 
            # both for simplex, as well as for ordered vertices

            nl_new = (identify_list[nl])[1]
            nr_new = (identify_list[nr])[1]



            lex_ordered_simp = [S0[nl_new], S0[nr_new]]
            int_ordered_simp = sorted([nl_new,nr_new])

            S1.append(sc.Simplex1D(int_ordered_simp, int_filt_value, filt_value, cross_vec,lex_ordered_simp))

            int_filt_value +=1
    return S1


In [ ]:

def periodic_filtration(points, max_alpha_square=float("inf"), a=1, b=1, c=1):
    """
    Outputs
        periodic_filt ... list of tuples, each tuple containing 
            1) simplex encoded as list of (periodically reduced) vertex numbers
            2) filtration value as float
        For example, periodic_filt[22] = ([8, 11], 0.0016)
        
        [Simplices_0, Simplices_1, Simplices_2, Simplices_3], where
        Simplices_0 ... list of 0-simplices encoded as Simplex0D objects
        Simplices_1 ... list of 1-simplices encoded as Simplex1D objects
        Simplices_2 ... list of 2-simplices encoded as Simplex2D objects
        Simplices_3 ... list of 3-simplices encoded as Simplex3D objects
    """
    
    """
    Input:
        a, b, c ... side lengths of cell [0,a] x [0,b] x [0,c]
        points  ... numpy array of size (N,3) containing N points in cell
        max_alpha_square   ... maximum alpha (squared) value of alpha-filtration
        
    Output:
        XXXXXX
    """
    N = points.shape[0]
    points_dup = torus_copy(points)
    
    # alpha-complex generation by gudhi
    alpha_complex = gd.AlphaComplex(points_dup)
    simplex_tree = alpha_complex.create_simplex_tree(max_alpha_square=max_alpha_square)
    filtration = simplex_tree.get_filtration()   
    
    # split unidentified simplices by dimension
    simp0, simp1, simp2, simp3 = dim_split(filtration)
    
 
    Simplices_2 = []
    Simplices_3 = []
    
    """
    I removed the connected component list because I want to generate this as a method later,
    not at initialisation. 
    However, this STILL NEEDS TO BE DONE!!!
    """
    
    
    
    # 0-Simplices ---------------------------------
    S0 = create_S0(simp0)
    identify_list = identification_list(S0_list=simp0, S0, eps = 1e-5)

    
    # 1-Simplices ---------------------------------
    S1 = create_S1(S1_list=simp1,S0=S0, alpha_complex=alpha_complex)

    
    
    # 2-Simplices ---------------------------------
    
    # for the 2-simplices we need
    # the vertices (as numbers from 1 to N, i.e. [1, 3, 6]), 
    # the integer filtration value (i.e. 24), 
    # the continuous filtration value (i.e. 0.583)
    # the boundary 1-simplices given by their integer filtration values (i.e. [12,15,20])
    
    for i in range(len(simp2)):
            simp, filt_value = simp2[i]

            # how can we find the correct 2-simplices?

            # if we have the vertices of the 2-simplex, we can look for 1-simplices which have two of these boundary points
            # since we only keep 2-simplices with their left-most point in the main cell
            # we can calculate the crossing vectors and then uniquly identify 2 out of 3 boundary elements

            # for the last edge, we have to 
            # - take the middle and the rightmost point,
            # - shift them such that the middle point is now in the main cell
            # - calculate the crossing vector of these shifted points
            # this is then the crossing vector of the last edge, and we can again identify
            # the correct 1-simplex using the vertices and the crossing vector
                
                
            [n1, n2, n3] = simp
            coord1 = alpha_complex.get_point(n1)
            coord2 = alpha_complex.get_point(n2)
            coord3 = alpha_complex.get_point(n3)
            
            # ordering vertices
            a0 = n1
            b0 = n2
            c0 = n3
            a0_coord = coord1
            b0_coord = coord2
            c0_coord = coord3
            
            a1, a1_coord, b1, b1_coord = order_vertex(a0, a0_coord, b0, b0_coord)
            a2, a2_coord, c1, c1_coord = order_vertex(a1, a1_coord, c0, c0_coord)
            b2, b2_coord, c2, c2_coord = order_vertex(b1, b1_coord, c1, c1_coord)
            # a2 is the leftmost, b2 is the middle and c2 is the rightmost coordinate, 
            # or a2 < b2 < c2 in lexicographical order
            
            
            # only if a2 is in the main cell do we continue
            if check_bound(a2_coord, xmax=a, ymax=b): 

                # the vertices are not yet in the naming convention we have chosen
                # so we rename them using identify_list
                for i in range(len(identify_list)):
                    new_name = (identify_list[i])[1]
                    if i == a2:
                        a2 = new_name
                    if i == b2:
                        b2 = new_name
                    if i == c2:
                        c2 = new_name


                # first boundary element from a2 to b2
                verts_1  = sorted([a2,b2])
                cr_vec_1 = crossing_vector(b2_coord,xmax=a,ymax=b)

                # second boundary element from a2 to c2
                verts_2  = sorted([a2,c2])
                cr_vec_2 = crossing_vector(c2_coord,xmax=a,ymax=b)

                # third boundary element from b2 to c2
                shift    = crossing_vector(b2_coord,xmax=a,ymax=b)
                verts_3  = sorted([b2,c2])
                cr_vec_3 = crossing_vector([c2_coord[0]-shift[0], c2_coord[1]-shift[1]])
                
                for j in range(len(Simplices_1)):
                    Simplex = Simplices_1[j]
                    verts = Simplex.verts
                    cv    = Simplex.cv
                    
                    
                    if (verts[0] == verts_1[0]) and (verts[1] == verts_1[1]) and (np.linalg.norm(cv-cr_vec_1)<eps):
                        bound_1 = Simplex
                    if (verts[0] == verts_2[0]) and (verts[1] == verts_2[1]) and (np.linalg.norm(cv-cr_vec_2)<eps):
                        bound_2 = Simplex
                    if (verts[0] == verts_3[0]) and (verts[1] == verts_3[1]) and (np.linalg.norm(cv-cr_vec_3)<eps):
                        bound_3 = Simplex

                Simplices_2.append(sc.Simplex2D( sorted([a2,b2,c2]), int_filt_value, filt_value, [Simplices_0[a2],Simplices_0[b2],Simplices_0[c2]], [bound_1,bound_2,bound_3]))
                int_filt_value += 1
                
    
   
    periodic_filt = []
    
    periodic_filt_0 = [(Simplex.verts, Simplex.cont_filt) for Simplex in Simplices_0]
    periodic_filt += periodic_filt_0
    
    periodic_filt_1 = [(Simplex.verts, Simplex.cont_filt) for Simplex in Simplices_1]
    periodic_filt += periodic_filt_1
    
    periodic_filt_2 = [(Simplex.verts, Simplex.cont_filt) for Simplex in Simplices_2]
    periodic_filt += periodic_filt_2
    
    periodic_filt_3 = [(Simplex.verts, Simplex.cont_filt) for Simplex in Simplices_3]
    periodic_filt += periodic_filt_3

    f = lambda some_tuple: some_tuple[1]
    periodic_filt.sort(key = f)
    
    
    # right now, the integer filtration value is wrong, because it is first sorted by dimension,
    # only afterwards by actual filtration value. 
    # periodic_filt tells us how many steps we actually have
    # and by searching for the index of (Simplex.verts, Simplex.cont_filt) in periodic_filt, 
    # we should get the integer filtration value
    
    for Simplex in Simplices_0:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    for Simplex in Simplices_1:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    for Simplex in Simplices_2:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    for Simplex in Simplices_3:
        int_filt = periodic_filt.index((Simplex.verts, Simplex.cont_filt))
        Simplex.update_int_filt(int_filt)
    
    
    return periodic_filt, [Simplices_0, Simplices_1, Simplices_2, Simplices_3]
    
